In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

**Load in Documentary scraping from BoxMojo** 

In [3]:
docu = pd.read_csv('./docu_movies_comprehensive.csv')
docu.head(3)

,Unnamed: 0,link_stub,rank,title,lifetime_gross,max_theaters,opening,open_th,release_date,distributor,domestic_total_gross,international_gross,worldwide_gross,budget,runtime,MPAA,Genres,Director,Writers,Producers
0,Fahrenheit 9/11,/title/tt0361596/?ref_=bo_ge_table_1,1,Fahrenheit 9/11,"$119,194,771","2,011","$23,920,637",868,"Jun 23, 2004",Lionsgate\n\n,119194771,103252111,222446882.0,6000000.0,122.0,R,"['Documentary', 'Drama', 'War']",['Michael Moore'],['Michael Moore'],"['Jim Czarnecki', 'Kathleen Glynn', 'Michael M..."
1,March of the Penguins,/title/tt0428803/?ref_=bo_ge_table_2,2,March of the Penguins,"$77,437,223","2,506","$137,492",4,"Jun 24, 2005",Warner Independent Pictures (WIP)\n\n,77437223,49955470,127392693.0,NaN,80.0,NaN,"['Documentary', 'Family']",['Luc Jacquet'],"['Luc Jacquet', 'Luc Jacquet', 'Michel Fessler...","['Yves Darondeau', 'Christophe Lioud', 'Emmanu..."
2,Justin Bieber: Never Say Never,/title/tt1702443/?ref_=bo_ge_table_3,3,Justin Bieber: Never Say Never,"$73,013,910","3,118","$29,514,054","3,105","Feb 11, 2011",Paramount Pictures\n\n,73013910,26022917,99036827.0,13000000.0,105.0,NaN,"['Documentary', 'Music']",['Jon M. Chu'],[],"['Justin Bieber', 'Scooter Braun', 'Dan Cutfor..."


In [4]:
docu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            1969 non-null   object 
 1   link_stub             1969 non-null   object 
 2   rank                  1969 non-null   object 
 3   title                 1969 non-null   object 
 4   lifetime_gross        1969 non-null   object 
 5   max_theaters          1969 non-null   object 
 6   opening               1969 non-null   object 
 7   open_th               1969 non-null   object 
 8   release_date          1969 non-null   object 
 9   distributor           1969 non-null   object 
 10  domestic_total_gross  1969 non-null   int64  
 11  international_gross   1969 non-null   int64  
 12  worldwide_gross       644 non-null    float64
 13  budget                30 non-null     float64
 14  runtime               1756 non-null   float64
 15  MPAA                 

In [5]:
docu["Director_Name"] = docu.Director.str[2:-2]
docu

,Unnamed: 0,link_stub,rank,title,lifetime_gross,max_theaters,opening,open_th,release_date,distributor,...,international_gross,worldwide_gross,budget,runtime,MPAA,Genres,Director,Writers,Producers,Director_Name
0,Fahrenheit 9/11,/title/tt0361596/?ref_=bo_ge_table_1,1,Fahrenheit 9/11,"$119,194,771","2,011","$23,920,637",868,"Jun 23, 2004",Lionsgate\n\n,...,103252111,222446882.0,6000000.0,122.0,R,"['Documentary', 'Drama', 'War']",['Michael Moore'],['Michael Moore'],"['Jim Czarnecki', 'Kathleen Glynn', 'Michael M...",Michael Moore
1,March of the Penguins,/title/tt0428803/?ref_=bo_ge_table_2,2,March of the Penguins,"$77,437,223","2,506","$137,492",4,"Jun 24, 2005",Warner Independent Pictures (WIP)\n\n,...,49955470,127392693.0,NaN,80.0,NaN,"['Documentary', 'Family']",['Luc Jacquet'],"['Luc Jacquet', 'Luc Jacquet', 'Michel Fessler...","['Yves Darondeau', 'Christophe Lioud', 'Emmanu...",Luc Jacquet
2,Justin Bieber: Never Say Never,/title/tt1702443/?ref_=bo_ge_table_3,3,Justin Bieber: Never Say Never,"$73,013,910","3,118","$29,514,054","3,105","Feb 11, 2011",Paramount Pictures\n\n,...,26022917,99036827.0,13000000.0,105.0,NaN,"['Documentary', 'Music']",['Jon M. Chu'],[],"['Justin Bieber', 'Scooter Braun', 'Dan Cutfor...",Jon M. Chu
3,This Is It,/title/tt1477715/?ref_=bo_ge_table_4,4,This Is It,"$72,091,016","3,481","$23,234,394","3,481","Oct 28, 2009",Sony Pictures Entertainment (SPE)\n\n,...,189092572,261183588.0,NaN,111.0,PG,"['Documentary', 'Drama', 'Music']",['Kenny Ortega'],[],[],Kenny Ortega
4,Under the Sea 3D,/title/tt1020876/?ref_=bo_ge_table_5,5,Under the Sea 3D,"$36,262,926",108,"$661,185",49,"Feb 13, 2009",Warner Bros.\n\n,...,19639698,55902624.0,NaN,NaN,NaN,"['Documentary', 'Short']",['Howard Hall'],"['Howard Hall', 'Toni Myers', 'Graeme Ferguson']",['Toni Myers'],Howard Hall
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,Patagonia Rising,/title/tt1820592/?ref_=bo_ge_table_95,"1,995",Patagonia Rising,"$2,064",1,$981,1,"Jun 8, 2012",First Run\n\n,...,2064,NaN,NaN,NaN,NaN,"['Adventure', 'Documentary', 'Western']",['Brian Lilla'],[],"['Scott Douglas', 'Greg Miller']",Brian Lilla
1965,Algorithms,/title/tt2616594/?ref_=bo_ge_table_96,"1,996",Algorithms,"$2,021",1,$317,1,"Oct 17, 2014",First Run\n\n,...,2021,NaN,NaN,NaN,NaN,"['Documentary', 'Sport']","['Ian McDonald', 'Geetha J']",[],['Geetha J'],"Ian McDonald', 'Geetha J"
1966,Split: A Deeper Divide,/title/tt2382420/?ref_=bo_ge_table_97,"1,997",Split: A Deeper Divide,"$2,000",1,"$2,338",1,"Oct 12, 2012",Feature Presentations Releasing\n\n,...,2000,NaN,150000.0,72.0,NaN,['Documentary'],['Kelly Nyks'],['Kelly Nyks'],"['Jeff Beard', 'Peter D. Hutchison', 'Kelly Ny...",Kelly Nyks
1967,"Accidental Courtesy: Daryl Davis, Race & America",/title/tt5390430/?ref_=bo_ge_table_98,"1,998","Accidental Courtesy: Daryl Davis, Race & America","$1,940",1,"$1,452",1,"Jan 6, 2017",First Run\n\n,...,1940,NaN,NaN,NaN,NaN,"['Biography', 'Documentary', 'History']",['Matthew Ornstein'],[],"['Matthew Ornstein', 'Noah Ornstein']",Matthew Ornstein


In [6]:
def get_url(director_name): 
    try: 
        director_words = director_name.split(" ")
        number_words = len(director_words)
        url = "https://en.wikipedia.org/wiki/"+director_words[0]
        for i in range(number_words-1): 
            url+="_"+director_words[i+1]
        return url 
    except: 
        return None
get_url("Michael Moore")

'https://en.wikipedia.org/wiki/Michael_Moore'

In [36]:
docu["Wikipedia_director"] = docu["Director_Name"].apply(lambda x: get_url(x)) 
docu.head(20)

,Unnamed: 0,link_stub,rank,title,lifetime_gross,max_theaters,opening,open_th,release_date,distributor,...,worldwide_gross,budget,runtime,MPAA,Genres,Director,Writers,Producers,Director_Name,Wikipedia_director
0,Fahrenheit 9/11,/title/tt0361596/?ref_=bo_ge_table_1,1,Fahrenheit 9/11,"$119,194,771","2,011","$23,920,637",868,"Jun 23, 2004",Lionsgate\n\n,...,222446882.0,6000000.0,122.0,R,"['Documentary', 'Drama', 'War']",['Michael Moore'],['Michael Moore'],"['Jim Czarnecki', 'Kathleen Glynn', 'Michael M...",Michael Moore,https://en.wikipedia.org/wiki/Michael_Moore
1,March of the Penguins,/title/tt0428803/?ref_=bo_ge_table_2,2,March of the Penguins,"$77,437,223","2,506","$137,492",4,"Jun 24, 2005",Warner Independent Pictures (WIP)\n\n,...,127392693.0,NaN,80.0,NaN,"['Documentary', 'Family']",['Luc Jacquet'],"['Luc Jacquet', 'Luc Jacquet', 'Michel Fessler...","['Yves Darondeau', 'Christophe Lioud', 'Emmanu...",Luc Jacquet,https://en.wikipedia.org/wiki/Luc_Jacquet
2,Justin Bieber: Never Say Never,/title/tt1702443/?ref_=bo_ge_table_3,3,Justin Bieber: Never Say Never,"$73,013,910","3,118","$29,514,054","3,105","Feb 11, 2011",Paramount Pictures\n\n,...,99036827.0,13000000.0,105.0,NaN,"['Documentary', 'Music']",['Jon M. Chu'],[],"['Justin Bieber', 'Scooter Braun', 'Dan Cutfor...",Jon M. Chu,https://en.wikipedia.org/wiki/Jon_M._Chu
3,This Is It,/title/tt1477715/?ref_=bo_ge_table_4,4,This Is It,"$72,091,016","3,481","$23,234,394","3,481","Oct 28, 2009",Sony Pictures Entertainment (SPE)\n\n,...,261183588.0,NaN,111.0,PG,"['Documentary', 'Drama', 'Music']",['Kenny Ortega'],[],[],Kenny Ortega,https://en.wikipedia.org/wiki/Kenny_Ortega
4,Under the Sea 3D,/title/tt1020876/?ref_=bo_ge_table_5,5,Under the Sea 3D,"$36,262,926",108,"$661,185",49,"Feb 13, 2009",Warner Bros.\n\n,...,55902624.0,NaN,NaN,NaN,"['Documentary', 'Short']",['Howard Hall'],"['Howard Hall', 'Toni Myers', 'Graeme Ferguson']",['Toni Myers'],Howard Hall,https://en.wikipedia.org/wiki/Howard_Hall
5,2016: Obama's America,/title/tt2247692/?ref_=bo_ge_table_6,6,2016: Obama's America,"$33,449,086","2,017","$31,610",1,"Jul 13, 2012",Rocky Mountain Pictures\n\n,...,NaN,NaN,87.0,PG,['Documentary'],"[""Dinesh D'Souza"", 'John Sullivan']","[""Dinesh D'Souza"", 'John Sullivan', ""Dinesh D'...","['Ann Balog', 'Doug Sain']","Dinesh D'Souza"", 'John Sullivan","https://en.wikipedia.org/wiki/Dinesh_D'Souza"",..."
6,Earth,/title/tt0393597/?ref_=bo_ge_table_7,7,Earth,"$32,011,576","1,804","$8,825,760","1,804","Apr 22, 2009",Walt Disney Studios Motion Pictures\n\n,...,108976456.0,NaN,90.0,NaN,['Documentary'],"['Alastair Fothergill', 'Mark Linfield']","['Alastair Fothergill', 'Mark Linfield', 'Lesl...","['Sophokles Tasioulis', 'Alix Tidmarsh']","Alastair Fothergill', 'Mark Linfield",https://en.wikipedia.org/wiki/Alastair_Fotherg...
7,Chimpanzee,/title/tt1222815/?ref_=bo_ge_table_8,8,Chimpanzee,"$28,972,764","1,567","$10,673,748","1,563","Apr 20, 2012",Walt Disney Studios Motion Pictures\n\n,...,34823764.0,NaN,78.0,G,"['Documentary', 'Family']","['Alastair Fothergill', 'Mark Linfield']","['Alastair Fothergill', 'Mark Linfield', 'Mark...","['Alastair Fothergill', 'Mark Linfield', 'Alix...","Alastair Fothergill', 'Mark Linfield",https://en.wikipedia.org/wiki/Alastair_Fotherg...
8,One Direction: This Is Us,/title/tt2515086/?ref_=bo_ge_table_9,9,One Direction: This Is Us,"$28,873,374","2,735","$15,815,497","2,735","Aug 30, 2013",TriStar Pictures\n\n,...,68567841.0,10000000.0,92.0,PG,"['Documentary', 'Music']",['Morgan Spurlock'],[],"['Simon Cowell', 'Ted Kenney', 'Adam Milano', ...",Morgan Spurlock,https://en.wikipedia.org/wiki/Morgan_Spurlock
9,Born to Be Wild,/title/tt1680059/?ref_=bo_ge_table_10,10,Born to Be Wild,"$25,978,556",208,"$871,478",206,"Apr 8, 2011",Warner Bros.\n\n,...,41644475.0,NaN,NaN,NaN,"['Adventure', 'Documentary', 'Short']",['David Lickley'],['Drew Fellman'],['Drew Fellman'],David Lickley,https://en.wikipedia.org/wiki/David_Lickley


In [8]:
docu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            1969 non-null   object 
 1   link_stub             1969 non-null   object 
 2   rank                  1969 non-null   object 
 3   title                 1969 non-null   object 
 4   lifetime_gross        1969 non-null   object 
 5   max_theaters          1969 non-null   object 
 6   opening               1969 non-null   object 
 7   open_th               1969 non-null   object 
 8   release_date          1969 non-null   object 
 9   distributor           1969 non-null   object 
 10  domestic_total_gross  1969 non-null   int64  
 11  international_gross   1969 non-null   int64  
 12  worldwide_gross       644 non-null    float64
 13  budget                30 non-null     float64
 14  runtime               1756 non-null   float64
 15  MPAA                 

In [9]:
docu.Wikipedia_director.value_counts()

https://en.wikipedia.org/wiki/                                     171
https://en.wikipedia.org/wiki/Michael_Moore                          9
https://en.wikipedia.org/wiki/Errol_Morris                           9
https://en.wikipedia.org/wiki/Alex_Gibney                            8
https://en.wikipedia.org/wiki/Steve_James                            7
                                                                  ... 
https://en.wikipedia.org/wiki/Matt_Embry                             1
https://en.wikipedia.org/wiki/Makoto_Satô                            1
https://en.wikipedia.org/wiki/Jim_Brown                              1
https://en.wikipedia.org/wiki/David_Sington                          1
https://en.wikipedia.org/wiki/John_Barbisan',_'Michael_Hamilton      1
Name: Wikipedia_director, Length: 1504, dtype: int64

**Check out Scrapability**

In [12]:
url = 'https://en.wikipedia.org/wiki/Kenny_Ortega' 

response = requests.get(url)
response.status_code

200

In [13]:
response.text[:1000] 


'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Kenny Ortega - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"bf88e8b0-375f-410f-8eea-47abbac564c8","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Kenny_Ortega","wgTitle":"Kenny Ortega","wgCurRevisionId":1046849125,"wgRevisionId":1046849125,"wgArticleId":3812712,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Pages with reference errors","Pages with broken reference names","Articles with short description","Short description is different from Wikidata","Use mdy dates fro

In [14]:
page = response.text
soup = BeautifulSoup(page, "lxml")

In [15]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Kenny Ortega - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"bf88e8b0-375f-410f-8eea-47abbac564c8","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Kenny_Ortega","wgTitle":"Kenny Ortega","wgCurRevisionId":1046849125,"wgRevisionId":1046849125,"wgArticleId":3812712,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Pages with reference errors","Pages with broken reference names","Articles with short description","Short description is different from Wikidata","Use m

In [16]:
for element in soup.find_all(class_= 'infobox-data'):
    print(element)

<td class="infobox-data"><div class="nickname" style="display:inline">Kenneth Miller Ortega</div><br/><span style="display:none"> (<span class="bday">1950-04-18</span>) </span>April 18, 1950<span class="noprint ForceAgeToShow"> (age 71)</span><br/><div class="birthplace" style="display:inline"><a href="/wiki/Palo_Alto,_California" title="Palo Alto, California">Palo Alto, California</a>, U.S.</div></td>
<td class="infobox-data role">Filmmaker, choreographer, touring manager</td>
<td class="infobox-data">1980–present</td>
<td class="infobox-data"><a href="/wiki/Billy_Squier" title="Billy Squier">Billy Squier</a>'s <i><a href="/wiki/Rock_Me_Tonite" title="Rock Me Tonite">Rock Me Tonite</a></i> video<br/><i><a href="/wiki/Newsies" title="Newsies">Newsies</a></i><br/><i><a href="/wiki/Hocus_Pocus_(1993_film)" title="Hocus Pocus (1993 film)">Hocus Pocus</a></i><br/><i><a href="/wiki/High_School_Musical" title="High School Musical">High School Musical</a></i><br/><i><a href="/wiki/Michael_Jac

**Decide what you want from soup**

*Name*

In [17]:
name = soup.find(class_= 'nickname').text
name

'Kenneth Miller Ortega'

*Birthday*

In [18]:
bday = soup.find(class_= 'bday').text
bday

'1950-04-18'

*Birthplace*

In [19]:
birthplace = soup.find(class_= 'birthplace').text
birthplace

'Palo Alto, California, U.S.'

**Build functions**

In [20]:
def get_name(soup): 
    try: 
        name = soup.find(class_= 'nickname').text
        return name
    except: 
        return None

In [21]:
def get_bday(soup): 
    try: 
        bday = soup.find(class_= 'bday').text
        return bday
    except: 
        return None

In [22]:
def get_birthplace(soup):
    try: 
        birthplace = soup.find(class_= 'birthplace').text
        return birthplace
    except: 
        return None

**Build pipeline**

In [31]:
def get_cast_info(url): 
    '''
    From the name of the cast member, request the html of the Wikipedia page 
    and using BeautifulSoup, get the Name, Birthday, Birthplace of the cast member. 
    Return all as dictionary. 
    '''
    try: 
        response = requests.get(url)
        cast = response.text
        soup = BeautifulSoup(cast, 'lxml')
    
        headers = ['cast_name', 'cast_bday', 'cast_bplace']

        #Get name 
        name = get_name(soup)

        #Get birthday
        bday = get_bday(soup)

        #Get birthplace 
        bplace = get_birthplace(soup)

        #Create dictionary for cast member and return
        member_dict = dict(zip(headers, [name, bday, bplace]))
        return member_dict
    except: 
        return  {'cast_name': "More Than 1 director", 'cast_bday': "More Than 1 director", 'cast_bplace': "More Than 1 director"}
    

Let's try this out

In [37]:
trial_directors = docu.iloc[0:12]

trial_directors_info = []

for link in trial_directors.Wikipedia_director:
    trial_directors_info.append(get_cast_info(link))

In [38]:
trial_directors_info

[{'cast_name': 'Michael Francis Moore',
  'cast_bday': '1954-04-23',
  'cast_bplace': 'Davison, Michigan, U.S.'},
 {'cast_name': None,
  'cast_bday': '1967-12-05',
  'cast_bplace': 'Bourg-en-Bresse, France'},
 {'cast_name': 'Jonathan Murray Chu',
  'cast_bday': '1979-11-02',
  'cast_bplace': 'Palo Alto, California, U.S.'},
 {'cast_name': 'Kenneth Miller Ortega',
  'cast_bday': '1950-04-18',
  'cast_bplace': 'Palo Alto, California, U.S.'},
 {'cast_name': None, 'cast_bday': None, 'cast_bplace': None},
 {'cast_name': None, 'cast_bday': None, 'cast_bplace': None},
 {'cast_name': None, 'cast_bday': None, 'cast_bplace': None},
 {'cast_name': None, 'cast_bday': None, 'cast_bplace': None},
 {'cast_name': 'Morgan Valentine Spurlock',
  'cast_bday': '1970-11-07',
  'cast_bplace': 'Parkersburg, West Virginia, U.S.'},
 {'cast_name': None, 'cast_bday': None, 'cast_bplace': None},
 {'cast_name': None, 'cast_bday': None, 'cast_bplace': None},
 {'cast_name': 'Michael Francis Moore',
  'cast_bday': '19

In [39]:
trial_directors_info = pd.DataFrame(trial_directors_info)  
trial_directors_info
# docu_movies_page_info.set_index('movie_title', inplace=True)

# docu_movies_page_info

,cast_name,cast_bday,cast_bplace
0,Michael Francis Moore,1954-04-23,"Davison, Michigan, U.S."
1,None,1967-12-05,"Bourg-en-Bresse, France"
2,Jonathan Murray Chu,1979-11-02,"Palo Alto, California, U.S."
3,Kenneth Miller Ortega,1950-04-18,"Palo Alto, California, U.S."
4,None,None,None
5,None,None,None
6,None,None,None
7,None,None,None
8,Morgan Valentine Spurlock,1970-11-07,"Parkersburg, West Virginia, U.S."
9,None,None,None


In [40]:
documentaries = trial_directors.merge(trial_directors_info, left_index=True, right_index=True)
documentaries

,Unnamed: 0,link_stub,rank,title,lifetime_gross,max_theaters,opening,open_th,release_date,distributor,...,MPAA,Genres,Director,Writers,Producers,Director_Name,Wikipedia_director,cast_name,cast_bday,cast_bplace
0,Fahrenheit 9/11,/title/tt0361596/?ref_=bo_ge_table_1,1,Fahrenheit 9/11,"$119,194,771","2,011","$23,920,637",868,"Jun 23, 2004",Lionsgate\n\n,...,R,"['Documentary', 'Drama', 'War']",['Michael Moore'],['Michael Moore'],"['Jim Czarnecki', 'Kathleen Glynn', 'Michael M...",Michael Moore,https://en.wikipedia.org/wiki/Michael_Moore,Michael Francis Moore,1954-04-23,"Davison, Michigan, U.S."
1,March of the Penguins,/title/tt0428803/?ref_=bo_ge_table_2,2,March of the Penguins,"$77,437,223","2,506","$137,492",4,"Jun 24, 2005",Warner Independent Pictures (WIP)\n\n,...,NaN,"['Documentary', 'Family']",['Luc Jacquet'],"['Luc Jacquet', 'Luc Jacquet', 'Michel Fessler...","['Yves Darondeau', 'Christophe Lioud', 'Emmanu...",Luc Jacquet,https://en.wikipedia.org/wiki/Luc_Jacquet,None,1967-12-05,"Bourg-en-Bresse, France"
2,Justin Bieber: Never Say Never,/title/tt1702443/?ref_=bo_ge_table_3,3,Justin Bieber: Never Say Never,"$73,013,910","3,118","$29,514,054","3,105","Feb 11, 2011",Paramount Pictures\n\n,...,NaN,"['Documentary', 'Music']",['Jon M. Chu'],[],"['Justin Bieber', 'Scooter Braun', 'Dan Cutfor...",Jon M. Chu,https://en.wikipedia.org/wiki/Jon_M._Chu,Jonathan Murray Chu,1979-11-02,"Palo Alto, California, U.S."
3,This Is It,/title/tt1477715/?ref_=bo_ge_table_4,4,This Is It,"$72,091,016","3,481","$23,234,394","3,481","Oct 28, 2009",Sony Pictures Entertainment (SPE)\n\n,...,PG,"['Documentary', 'Drama', 'Music']",['Kenny Ortega'],[],[],Kenny Ortega,https://en.wikipedia.org/wiki/Kenny_Ortega,Kenneth Miller Ortega,1950-04-18,"Palo Alto, California, U.S."
4,Under the Sea 3D,/title/tt1020876/?ref_=bo_ge_table_5,5,Under the Sea 3D,"$36,262,926",108,"$661,185",49,"Feb 13, 2009",Warner Bros.\n\n,...,NaN,"['Documentary', 'Short']",['Howard Hall'],"['Howard Hall', 'Toni Myers', 'Graeme Ferguson']",['Toni Myers'],Howard Hall,https://en.wikipedia.org/wiki/Howard_Hall,None,None,None
5,2016: Obama's America,/title/tt2247692/?ref_=bo_ge_table_6,6,2016: Obama's America,"$33,449,086","2,017","$31,610",1,"Jul 13, 2012",Rocky Mountain Pictures\n\n,...,PG,['Documentary'],"[""Dinesh D'Souza"", 'John Sullivan']","[""Dinesh D'Souza"", 'John Sullivan', ""Dinesh D'...","['Ann Balog', 'Doug Sain']","Dinesh D'Souza"", 'John Sullivan","https://en.wikipedia.org/wiki/Dinesh_D'Souza"",...",None,None,None
6,Earth,/title/tt0393597/?ref_=bo_ge_table_7,7,Earth,"$32,011,576","1,804","$8,825,760","1,804","Apr 22, 2009",Walt Disney Studios Motion Pictures\n\n,...,NaN,['Documentary'],"['Alastair Fothergill', 'Mark Linfield']","['Alastair Fothergill', 'Mark Linfield', 'Lesl...","['Sophokles Tasioulis', 'Alix Tidmarsh']","Alastair Fothergill', 'Mark Linfield",https://en.wikipedia.org/wiki/Alastair_Fotherg...,None,None,None
7,Chimpanzee,/title/tt1222815/?ref_=bo_ge_table_8,8,Chimpanzee,"$28,972,764","1,567","$10,673,748","1,563","Apr 20, 2012",Walt Disney Studios Motion Pictures\n\n,...,G,"['Documentary', 'Family']","['Alastair Fothergill', 'Mark Linfield']","['Alastair Fothergill', 'Mark Linfield', 'Mark...","['Alastair Fothergill', 'Mark Linfield', 'Alix...","Alastair Fothergill', 'Mark Linfield",https://en.wikipedia.org/wiki/Alastair_Fotherg...,None,None,None
8,One Direction: This Is Us,/title/tt2515086/?ref_=bo_ge_table_9,9,One Direction: This Is Us,"$28,873,374","2,735","$15,815,497","2,735","Aug 30, 2013",TriStar Pictures\n\n,...,PG,"['Documentary', 'Music']",['Morgan Spurlock'],[],"['Simon Cowell', 'Ted Kenney', 'Adam Milano', ...",Morgan Spurlock,https://en.wikipedia.org/wiki/Morgan_Spurlock,Morgan Valentine Spurlock,1970-11-07,"Parkersburg, West Virginia, U.S."
9,Born to Be Wild,/title/tt1680059/?ref_=bo_ge_table_10,10,Born to Be Wild,"$25,978,556",208,"$871,478",206,"Apr 8, 2011",Warner Bros.\n\n,...,NaN,"['Adventure', 'Documentary', 'Short']",['David Lickley'],['Drew 

**Now onto the whole dataframe**

In [41]:
docu_directors_info = []

for link in docu.Wikipedia_director:
    docu_directors_info.append(get_cast_info(link))

In [42]:
docu_directors_info = pd.DataFrame(docu_directors_info)  

In [43]:
documentaries = docu.merge(docu_directors_info, left_index=True, right_index=True)
documentaries

,Unnamed: 0,link_stub,rank,title,lifetime_gross,max_theaters,opening,open_th,release_date,distributor,...,MPAA,Genres,Director,Writers,Producers,Director_Name,Wikipedia_director,cast_name,cast_bday,cast_bplace
0,Fahrenheit 9/11,/title/tt0361596/?ref_=bo_ge_table_1,1,Fahrenheit 9/11,"$119,194,771","2,011","$23,920,637",868,"Jun 23, 2004",Lionsgate\n\n,...,R,"['Documentary', 'Drama', 'War']",['Michael Moore'],['Michael Moore'],"['Jim Czarnecki', 'Kathleen Glynn', 'Michael M...",Michael Moore,https://en.wikipedia.org/wiki/Michael_Moore,Michael Francis Moore,1954-04-23,"Davison, Michigan, U.S."
1,March of the Penguins,/title/tt0428803/?ref_=bo_ge_table_2,2,March of the Penguins,"$77,437,223","2,506","$137,492",4,"Jun 24, 2005",Warner Independent Pictures (WIP)\n\n,...,NaN,"['Documentary', 'Family']",['Luc Jacquet'],"['Luc Jacquet', 'Luc Jacquet', 'Michel Fessler...","['Yves Darondeau', 'Christophe Lioud', 'Emmanu...",Luc Jacquet,https://en.wikipedia.org/wiki/Luc_Jacquet,None,1967-12-05,"Bourg-en-Bresse, France"
2,Justin Bieber: Never Say Never,/title/tt1702443/?ref_=bo_ge_table_3,3,Justin Bieber: Never Say Never,"$73,013,910","3,118","$29,514,054","3,105","Feb 11, 2011",Paramount Pictures\n\n,...,NaN,"['Documentary', 'Music']",['Jon M. Chu'],[],"['Justin Bieber', 'Scooter Braun', 'Dan Cutfor...",Jon M. Chu,https://en.wikipedia.org/wiki/Jon_M._Chu,Jonathan Murray Chu,1979-11-02,"Palo Alto, California, U.S."
3,This Is It,/title/tt1477715/?ref_=bo_ge_table_4,4,This Is It,"$72,091,016","3,481","$23,234,394","3,481","Oct 28, 2009",Sony Pictures Entertainment (SPE)\n\n,...,PG,"['Documentary', 'Drama', 'Music']",['Kenny Ortega'],[],[],Kenny Ortega,https://en.wikipedia.org/wiki/Kenny_Ortega,Kenneth Miller Ortega,1950-04-18,"Palo Alto, California, U.S."
4,Under the Sea 3D,/title/tt1020876/?ref_=bo_ge_table_5,5,Under the Sea 3D,"$36,262,926",108,"$661,185",49,"Feb 13, 2009",Warner Bros.\n\n,...,NaN,"['Documentary', 'Short']",['Howard Hall'],"['Howard Hall', 'Toni Myers', 'Graeme Ferguson']",['Toni Myers'],Howard Hall,https://en.wikipedia.org/wiki/Howard_Hall,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,Patagonia Rising,/title/tt1820592/?ref_=bo_ge_table_95,"1,995",Patagonia Rising,"$2,064",1,$981,1,"Jun 8, 2012",First Run\n\n,...,NaN,"['Adventure', 'Documentary', 'Western']",['Brian Lilla'],[],"['Scott Douglas', 'Greg Miller']",Brian Lilla,https://en.wikipedia.org/wiki/Brian_Lilla,None,None,None
1965,Algorithms,/title/tt2616594/?ref_=bo_ge_table_96,"1,996",Algorithms,"$2,021",1,$317,1,"Oct 17, 2014",First Run\n\n,...,NaN,"['Documentary', 'Sport']","['Ian McDonald', 'Geetha J']",[],['Geetha J'],"Ian McDonald', 'Geetha J","https://en.wikipedia.org/wiki/Ian_McDonald',_'...",None,None,None
1966,Split: A Deeper Divide,/title/tt2382420/?ref_=bo_ge_table_97,"1,997",Split: A Deeper Divide,"$2,000",1,"$2,338",1,"Oct 12, 2012",Feature Presentations Releasing\n\n,...,NaN,['Documentary'],['Kelly Nyks'],['Kelly Nyks'],"['Jeff Beard', 'Peter D. Hutchison', 'Kelly Ny...",Kelly Nyks,https://en.wikipedia.org/wiki/Kelly_Nyks,None,None,None
1967,"Accidental Courtesy: Daryl Davis, Race & America",/title/tt5390430/?ref_=bo_ge_table_98,"1,998","Accidental Courtesy: Daryl Davis, Race & America","$1,940",1,"$1,452",1,"Jan 6, 2017",First Run\n\n,...,NaN,"['Biography', 'Documentary', 'History']",['Matthew Ornstein'],[],"['Matthew Ornstein', 'Noah Ornstein']",Matthew Ornstein,https://en.wikipedia.org/wiki/Matthew_Ornstein,None,None,None


In [44]:
documentaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            1969 non-null   object 
 1   link_stub             1969 non-null   object 
 2   rank                  1969 non-null   object 
 3   title                 1969 non-null   object 
 4   lifetime_gross        1969 non-null   object 
 5   max_theaters          1969 non-null   object 
 6   opening               1969 non-null   object 
 7   open_th               1969 non-null   object 
 8   release_date          1969 non-null   object 
 9   distributor           1969 non-null   object 
 10  domestic_total_gross  1969 non-null   int64  
 11  international_gross   1969 non-null   int64  
 12  worldwide_gross       644 non-null    float64
 13  budget                30 non-null     float64
 14  runtime               1756 non-null   float64
 15  MPAA                 

In [45]:
documentaries.to_csv('docu_movies_comprehensive_with_wikis.csv')